# **Setup**

In [ ]:
!pip install -qq langchain langchain-core langchain-google-genai langchain-community


In [2]:
import os

In [3]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [4]:
os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY


In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3,google_api_key=GEMINI_API_KEY)

# **CREATE SAMPLE DATAFRAMES**

In [7]:
import pandas as pd

In [8]:
df_sales = pd.DataFrame({
    "order_id": [1, 2, 3, 4],
    "product": ["Shoes", "Shirt", "Hat", "Shoes"],
    "amount": [100, 40, 25, 120]
})

df_customers = pd.DataFrame({
    "customer_id": [101, 102, 103, 104],
    "name": ["Alice", "Bob", "Charlie", "David"],
    "city": ["NY", "LA", "NY", "SF"]
})

print("Sales DataFrame:\n", df_sales)
print("\nCustomers DataFrame:\n", df_customers)

Sales DataFrame:
    order_id product  amount
0         1   Shoes     100
1         2   Shirt      40
2         3     Hat      25
3         4   Shoes     120

Customers DataFrame:
    customer_id     name city
0          101    Alice   NY
1          102      Bob   LA
2          103  Charlie   NY
3          104    David   SF


# **MULTI-DATAFRAME AGENT**

In [12]:
!pip install -qq langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 3.9 MB/s eta 0:00:00


In [13]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

In [17]:
agent = create_pandas_dataframe_agent(
    llm,
    [df_sales, df_customers],     # <-- pass multiple with names
    verbose=True,
    allow_dangerous_code=True
)

# **EXAMPLES**

In [19]:
print("\n--- Query 1 ---")
response1 = agent.invoke("What is the total sales amount?")
print("Answer:", response1["output"])


--- Query 1 ---


> Entering new AgentExecutor chain...
Thought: I need to sum the 'amount' column in df1.
Action: python_repl_ast
Action Input: 
```python
import pandas as pd
data1 = {'order_id': [1, 2, 3, 4], 'product': ['Shoes', 'Shirt', 'Hat', 'Shoes'], 'amount': [100, 40, 25, 120]}
df1 = pd.DataFrame(data1)
print(df1['amount'].sum())
```285
Thought: I now know the final answer.
Final Answer: 285


> Finished chain.
Answer: 285


In [20]:
print("\n--- Query 2 ---")
response2 = agent.invoke("Which products are sold?")
print("Answer:", response2["output"])


--- Query 2 ---


> Entering new AgentExecutor chain...
Thought: I need to extract the unique product names from the `df1` DataFrame.
Action: python_repl_ast
Action Input: ```python
import pandas as pd
data1 = {'order_id': [1, 2, 3, 4], 'product': ['Shoes', 'Shirt', 'Hat', 'Shoes'], 'amount': [100, 40, 25, 120]}
df1 = pd.DataFrame(data1)
print(df1['product'].unique())
```['Shoes' 'Shirt' 'Hat']
Thought: I now know the final answer.
Final Answer: The products sold are Shoes, Shirt, and Hat.


> Finished chain.
Answer: The products sold are Shoes, Shirt, and Hat.


In [21]:
print("\n--- Query 3 ---")
response3 = agent.invoke("List the cities where customers are located.")
print("Answer:", response3["output"])


--- Query 3 ---


> Entering new AgentExecutor chain...
Thought: I need to extract the 'city' column from df2 and get a list of unique cities.

Action: python_repl_ast
Action Input: 
```python
import pandas as pd
df2 = pd.DataFrame({'customer_id': [101, 102, 103, 104], 'name': ['Alice', 'Bob', 'Charlie', 'David'], 'city': ['NY', 'LA', 'NY', 'SF']})
cities = df2['city'].unique().tolist()
print(cities)
```
['NY', 'LA', 'SF']
I now know the final answer.
Final Answer: ['NY', 'LA', 'SF']


> Finished chain.
Answer: ['NY', 'LA', 'SF']


In [23]:
response4 = agent.invoke("How many rows are there in the df1?")
print(response4['output'])



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows in the dataframe df1.  I can do this using the `.shape` attribute.
Action: python_repl_ast
Action Input: 
```python
import pandas as pd
data1 = {'order_id': [1, 2, 3, 4], 'product': ['Shoes', 'Shirt', 'Hat', 'Shoes'], 'amount': [100, 40, 25, 120]}
df1 = pd.DataFrame(data1)
print(df1.shape[0])
```4
I now know the final answer
Final Answer: 4


> Finished chain.
4
